In [1]:
import pandas as pd
import os
import numpy as np
from pandas.util.testing import assert_frame_equal

In [3]:
normed = pd.read_table(
    '/projects/ps-yeolab3/bay001/maps/current/se/204_01_RBFOX2.merged.r2.1.HepG2_native_cassette_exons_all.normed_matrix.txt',
    sep=',',
    index_col=0
)
print(normed.shape)
normed.head()

(1805, 1400)


,0,1,2,3,4,5,6,7,8,9,...,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399
"chr10|+|101507147-101510125|101510153-101514285|101507147-101514285\t101507013-101507147\t101510125-101510153\t101514285-101514391\t1970,318\t18615,2970",-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,-0.000449,0.000266,0.000266,...,0.000133,0.000133,0.000133,0.000133,0.000133,0.000133,0.000133,0.000133,0.000133,0.000133
"chr10|+|103113985-103190101|103190209-103221737|103113985-103221737\t103113819-103113985\t103190101-103190209\t103221737-103221815\t1339,428\t10040,3008",-0.000092,0.000404,0.000404,0.000404,0.000404,0.000404,0.000900,0.000900,0.000900,0.000900,...,0.000404,0.000404,0.000404,-0.000092,-0.000092,-0.000092,-0.000092,-0.000092,-0.000092,-0.000092
"chr10|+|104629602-104629840|104629968-104632204|104629602-104632204\t104629561-104629602\t104629840-104629968\t104632204-104632355\t2366,322\t20222,2808",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000053,...,-0.000053,-0.000053,-0.000053,-0.000053,-0.000053,-0.000053,-0.000053,-0.000053,-0.000053,-0.000053
"chr10|+|104828479-104831530|104831596-104835842|104828479-104835842\t104828385-104828479\t104831530-104831596\t104835842-104836027\t584,1662\t5067,11184",-0.000110,-0.000110,-0.000110,-0.000110,-0.000110,-0.000110,-0.000110,-0.000110,-0.000110,-0.000110,...,-0.000110,-0.000110,-0.000110,-0.000110,-0.000110,-0.000110,-0.000110,-0.000110,-0.000110,-0.000110
"chr10|+|111890244-111892062|111892158-111893083|111890244-111893083\t111890120-111890244\t111892062-111892158\t111893083-111893969\t1877,1526\t19425,11386",-0.000125,-0.000125,-0.000125,-0.000125,-0.000125,-0.000125,-0.000125,-0.000125,-0.000125,-0.000125,...,-0.000125,-0.000125,-0.000125,-0.000125,-0.000125,-0.000125,-0.000125,-0.000125,-0.000125,-0.000125


In [11]:
def get_means_and_sems(df, conf=0.95):
    """
    Sets the means and standard error values after outlier
    removal. Replaces remove_outliers.
    Parameters
    ----------
    df : pandas.DataFrame
        table of densities or values
    conf : float
        keep {conf}% of densities present at every given position
    Returns
    -------
    means : list
        mean value for each position in the dataframe df
    sems : list
        standard error of the mean
    std_deviation: list
        standard deviation of the mean
    merged : pandas.DataFrame
        dataframe 'masked' of outliers
    """

    means = []
    sems = []
    std_deviation = []
    
    merged = pd.DataFrame(index=df.index)
    for key, value in df.iteritems():
        single_col = df[key].dropna()
        single_col = single_col.sort_values()
        nums = len(single_col)
        droppercent = (1 - conf) / 2.0
        dropnum = int(nums * droppercent)
        print("Num dropped: {}".format(dropnum))
        if dropnum > 0:
            print("number of pre-dropped values: {}".format(len(single_col)))
            single_col = single_col[dropnum:-dropnum]
            print("number of post-dropped values: {}".format(len(single_col)))
        merged = pd.merge(
            merged, pd.DataFrame(single_col), how='left',
            left_index=True, right_index=True
        )
        means.append(single_col.mean())
        sems.append(single_col.sem())
        std_deviation.append(single_col.std())

    return means, sems, std_deviation, merged

In [12]:
means, sems, std, merged = get_means_and_sems(normed)

Num dropped: 42
number of pre-dropped values: 1708
number of post-dropped values: 1624
Num dropped: 42
number of pre-dropped values: 1712
number of post-dropped values: 1628
Num dropped: 43
number of pre-dropped values: 1721
number of post-dropped values: 1635
Num dropped: 43
number of pre-dropped values: 1725
number of post-dropped values: 1639
Num dropped: 43
number of pre-dropped values: 1728
number of post-dropped values: 1642
Num dropped: 43
number of pre-dropped values: 1739
number of post-dropped values: 1653
Num dropped: 43
number of pre-dropped values: 1747
number of post-dropped values: 1661
Num dropped: 43
number of pre-dropped values: 1751
number of post-dropped values: 1665
Num dropped: 43
number of pre-dropped values: 1753
number of post-dropped values: 1667
Num dropped: 43
number of pre-dropped values: 1756
number of post-dropped values: 1670
Num dropped: 43
number of pre-dropped values: 1758
number of post-dropped values: 1672
Num dropped: 44
number of pre-dropped value

Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped value

Num dropped: 43
number of pre-dropped values: 1747
number of post-dropped values: 1661
Num dropped: 43
number of pre-dropped values: 1746
number of post-dropped values: 1660
Num dropped: 43
number of pre-dropped values: 1746
number of post-dropped values: 1660
Num dropped: 43
number of pre-dropped values: 1745
number of post-dropped values: 1659
Num dropped: 43
number of pre-dropped values: 1745
number of post-dropped values: 1659
Num dropped: 43
number of pre-dropped values: 1745
number of post-dropped values: 1659
Num dropped: 43
number of pre-dropped values: 1744
number of post-dropped values: 1658
Num dropped: 43
number of pre-dropped values: 1744
number of post-dropped values: 1658
Num dropped: 43
number of pre-dropped values: 1744
number of post-dropped values: 1658
Num dropped: 43
number of pre-dropped values: 1744
number of post-dropped values: 1658
Num dropped: 43
number of pre-dropped values: 1744
number of post-dropped values: 1658
Num dropped: 43
number of pre-dropped value

Num dropped: 41
number of pre-dropped values: 1662
number of post-dropped values: 1580
Num dropped: 41
number of pre-dropped values: 1662
number of post-dropped values: 1580
Num dropped: 41
number of pre-dropped values: 1660
number of post-dropped values: 1578
Num dropped: 41
number of pre-dropped values: 1656
number of post-dropped values: 1574
Num dropped: 41
number of pre-dropped values: 1656
number of post-dropped values: 1574
Num dropped: 41
number of pre-dropped values: 1655
number of post-dropped values: 1573
Num dropped: 41
number of pre-dropped values: 1655
number of post-dropped values: 1573
Num dropped: 41
number of pre-dropped values: 1654
number of post-dropped values: 1572
Num dropped: 41
number of pre-dropped values: 1651
number of post-dropped values: 1569
Num dropped: 41
number of pre-dropped values: 1648
number of post-dropped values: 1566
Num dropped: 41
number of pre-dropped values: 1647
number of post-dropped values: 1565
Num dropped: 41
number of pre-dropped value

Num dropped: 42
number of pre-dropped values: 1682
number of post-dropped values: 1598
Num dropped: 42
number of pre-dropped values: 1683
number of post-dropped values: 1599
Num dropped: 42
number of pre-dropped values: 1685
number of post-dropped values: 1601
Num dropped: 42
number of pre-dropped values: 1685
number of post-dropped values: 1601
Num dropped: 42
number of pre-dropped values: 1686
number of post-dropped values: 1602
Num dropped: 42
number of pre-dropped values: 1688
number of post-dropped values: 1604
Num dropped: 42
number of pre-dropped values: 1689
number of post-dropped values: 1605
Num dropped: 42
number of pre-dropped values: 1690
number of post-dropped values: 1606
Num dropped: 42
number of pre-dropped values: 1690
number of post-dropped values: 1606
Num dropped: 42
number of pre-dropped values: 1690
number of post-dropped values: 1606
Num dropped: 42
number of pre-dropped values: 1692
number of post-dropped values: 1608
Num dropped: 42
number of pre-dropped value

...

Num dropped: 41
number of pre-dropped values: 1655
number of post-dropped values: 1573
Num dropped: 41
number of pre-dropped values: 1654
number of post-dropped values: 1572
Num dropped: 41
number of pre-dropped values: 1653
number of post-dropped values: 1571
Num dropped: 41
number of pre-dropped values: 1652
number of post-dropped values: 1570
Num dropped: 41
number of pre-dropped values: 1652
number of post-dropped values: 1570
Num dropped: 41
number of pre-dropped values: 1651
number of post-dropped values: 1569
Num dropped: 41
number of pre-dropped values: 1651
number of post-dropped values: 1569
Num dropped: 41
number of pre-dropped values: 1651
number of post-dropped values: 1569
Num dropped: 41
number of pre-dropped values: 1651
number of post-dropped values: 1569
Num dropped: 41
number of pre-dropped values: 1651
number of post-dropped values: 1569
Num dropped: 41
number of pre-dropped values: 1651
number of post-dropped values: 1569
Num dropped: 41
number of pre-dropped value

Num dropped: 41
number of pre-dropped values: 1664
number of post-dropped values: 1582
Num dropped: 41
number of pre-dropped values: 1664
number of post-dropped values: 1582
Num dropped: 41
number of pre-dropped values: 1664
number of post-dropped values: 1582
Num dropped: 41
number of pre-dropped values: 1664
number of post-dropped values: 1582
Num dropped: 41
number of pre-dropped values: 1665
number of post-dropped values: 1583
Num dropped: 41
number of pre-dropped values: 1668
number of post-dropped values: 1586
Num dropped: 41
number of pre-dropped values: 1671
number of post-dropped values: 1589
Num dropped: 41
number of pre-dropped values: 1671
number of post-dropped values: 1589
Num dropped: 41
number of pre-dropped values: 1671
number of post-dropped values: 1589
Num dropped: 41
number of pre-dropped values: 1671
number of post-dropped values: 1589
Num dropped: 41
number of pre-dropped values: 1671
number of post-dropped values: 1589
Num dropped: 41
number of pre-dropped value

Num dropped: 43
number of pre-dropped values: 1733
number of post-dropped values: 1647
Num dropped: 43
number of pre-dropped values: 1733
number of post-dropped values: 1647
Num dropped: 43
number of pre-dropped values: 1734
number of post-dropped values: 1648
Num dropped: 43
number of pre-dropped values: 1735
number of post-dropped values: 1649
Num dropped: 43
number of pre-dropped values: 1735
number of post-dropped values: 1649
Num dropped: 43
number of pre-dropped values: 1736
number of post-dropped values: 1650
Num dropped: 43
number of pre-dropped values: 1737
number of post-dropped values: 1651
Num dropped: 43
number of pre-dropped values: 1738
number of post-dropped values: 1652
Num dropped: 43
number of pre-dropped values: 1738
number of post-dropped values: 1652
Num dropped: 43
number of pre-dropped values: 1741
number of post-dropped values: 1655
Num dropped: 43
number of pre-dropped values: 1741
number of post-dropped values: 1655
Num dropped: 43
number of pre-dropped value

Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped values: 1805
number of post-dropped values: 1715
Num dropped: 45
number of pre-dropped value

Num dropped: 43
number of pre-dropped values: 1747
number of post-dropped values: 1661
Num dropped: 43
number of pre-dropped values: 1742
number of post-dropped values: 1656
Num dropped: 43
number of pre-dropped values: 1739
number of post-dropped values: 1653
Num dropped: 43
number of pre-dropped values: 1732
number of post-dropped values: 1646
Num dropped: 43
number of pre-dropped values: 1726
number of post-dropped values: 1640
Num dropped: 43
number of pre-dropped values: 1724
number of post-dropped values: 1638


In [22]:
normed[['0']].dropna()['0'].sort_values().to_csv('/home/bay001/scratch/204_01_RBFOX2.merged.r2.1.HepG2_native_cassette_exons_all.normed_matrix.value0.txt')

In [23]:
normed[['0']].dropna()['0'].sort_values().shape

(1708,)

In [29]:
vals = normed[['0']].dropna()['0'].sort_values()
print(vals.head())
print(vals.tail())

chr2|-|216256537-216257653|216257926-216259250|216256537-216259250\t216256354-216256537\t216257653-216257926\t216259250-216259442\t100279,650249\t898227,4594658   -0.005570
chr20|-|47770608-47790731|47790806-47804652|47770608-47804652\t47770469-47770608\t47790731-47790806\t47804652-47804904\t21750,641\t149492,4660                     -0.004317
chr4|-|83280792-83292680|83292737-83294598|83280792-83294598\t83280623-83280792\t83292680-83292737\t83294598-83295139\t50503,9946\t379449,60482                    -0.003314
chr4|+|88901278-88901544|88901586-88902626|88901278-88902626\t88901197-88901278\t88901544-88901586\t88902626-88902950\t24782,5352\t199675,33824                    -0.002992
chr11|+|129992408-129993506|129993674-129996594|129992408-129996594\t129992199-129992408\t129993506-129993674\t129996594-129996725\t22750,40330\t202518,363388     -0.002752
Name: 0, dtype: float64
chr18|-|694334-697239|697355-706469|694334-706469\t694247-694334\t697239-697355\t706469-706578\t2091,178\t19530

In [25]:
vals[42:-42].shape

(1624,)

In [30]:
print(vals[1:-1].head())
print(vals[1:-1].tail())

chr20|-|47770608-47790731|47790806-47804652|47770608-47804652\t47770469-47770608\t47790731-47790806\t47804652-47804904\t21750,641\t149492,4660                   -0.004317
chr4|-|83280792-83292680|83292737-83294598|83280792-83294598\t83280623-83280792\t83292680-83292737\t83294598-83295139\t50503,9946\t379449,60482                  -0.003314
chr4|+|88901278-88901544|88901586-88902626|88901278-88902626\t88901197-88901278\t88901544-88901586\t88902626-88902950\t24782,5352\t199675,33824                  -0.002992
chr11|+|129992408-129993506|129993674-129996594|129992408-129996594\t129992199-129992408\t129993506-129993674\t129996594-129996725\t22750,40330\t202518,363388   -0.002752
chr7|+|99950893-99950995|99951635-99952765|99950893-99952765\t99950832-99950893\t99950995-99951635\t99952765-99952863\t2675,330\t28124,3300                      -0.002750
Name: 0, dtype: float64
chr16|+|8722967-8728895|8728936-8729024|8722967-8729024\t8722586-8722967\t8728895-8728936\t8729024-8729169\t1749,687\t154